# Student Name: VIGNESH GANESAN
## Student ID: 23079924

**Task 1**

In [1]:
import re
from collections import Counter

In [2]:
def normalizeText(text):
  return re.findall(r'\b\w+\b', text.lower())

def prepareMetadata(trainingSet):
  megaDocGB = ''
  megaDocIE = ''
  class_word_counts = {}
  class_counts = {}
  gb_bow = {}
  ie_bow = {}

  for doc, label in trainingSet:
    words = normalizeText(doc)
    if label == 'GB':
      megaDocGB += ' '.join(words)
      megaDocGB += ' '
    elif label =='IE':
      megaDocIE += ' '.join(words)
      megaDocIE += ' '

    for word in words:
      class_word_counts.setdefault(label, {})
      class_word_counts[label][word] = class_word_counts[label].get(word, 0) + 1

      class_counts.setdefault(label, 0)
      class_counts[label] += 1

  megaDocGB_split = megaDocGB.split()
  megaDocIE_split = megaDocIE.split()
  gb_bow = [megaDocGB_split.count(w) for w in megaDocGB_split]
  ie_bow = [megaDocIE_split.count(w) for w in megaDocIE_split]
  gb_bow = dict(zip(megaDocGB_split, gb_bow))
  ie_bow = dict(zip(megaDocIE_split, ie_bow))

  V = dict(Counter(gb_bow) + Counter(ie_bow))

  print('megaDocGB: ', ' '.join(normalizeText(megaDocGB)))
  print('megaDocIE: ', ' '.join(normalizeText(megaDocIE)))
  print()

  probGB = class_counts['GB'] / (class_counts['GB'] + class_counts['IE'])
  probIE = class_counts['IE'] / (class_counts['GB'] + class_counts['IE'])
  print('probGB: ', probGB)
  print('probIE: ', probIE)
  print()

  print('GB_BOW: ', gb_bow)
  print('IE_BOW: ', ie_bow)
  print()

  print('V: ', V)
  print('|V|: ', len(V.keys()))
  print()

  return gb_bow, ie_bow, V, class_counts, class_word_counts


def naiveBayesClassifier(trainingSet, testSet):
    gb_bow, ie_bow, V, class_counts, class_word_counts = prepareMetadata(trainingSet)
    total_docs = len(V)

    for test_doc, _ in testSet:
      print('Test document: ', test_doc)
      docProbGB = 0
      docProbIE = 0
      words = normalizeText(test_doc)
      for word in words:
        wordConditionalProbGB = gb_bow.get(word, 0) / len(V)
        wordConditionalProbIE = ie_bow.get(word, 0) / len(V)
        print(f"Word: '{word}' \t wordConditionalProbGB: {wordConditionalProbGB} \t wordConditionalProbIE: {wordConditionalProbIE}")

        word_count = class_word_counts.get('GB', {}).get(word, 0) + 1
        class_word_count = class_counts.get('GB', 0) + total_docs
        docProbGB += (word_count / class_word_count)

        word_count = class_word_counts.get('IE', {}).get(word, 0) + 1
        class_word_count = class_counts.get('IE', 0) + total_docs
        docProbIE += (word_count / class_word_count)
      print(f"docProbGB = {docProbGB} \t docProbIE = {docProbIE}")

      inferred_class = "GB" if docProbGB > docProbIE else "IE" if docProbGB < docProbIE else "NEUTRAL"
      print('Inferred Class: ', inferred_class)
      print('-'*50)

trainingSet = [
    ('London is the Capital of GB', 'GB'),
    ('Oxford is a city in GB', 'GB'),
    ('Dublin is the capital of Ireland', 'IE'),
    ('Limerick is a city in Ireland', 'IE')
]
testSet = [
    ('University of Limerick', '?'),
    ('University College Dublin', '?'),
    ('Imperial College London', '?'),
    ('University of Oxford', '?'),
    ('Ireland & GB', '?')
]

naiveBayesClassifier(trainingSet, testSet)

megaDocGB:  london is the capital of gb oxford is a city in gb
megaDocIE:  dublin is the capital of ireland limerick is a city in ireland

probGB:  0.5
probIE:  0.5

GB_BOW:  {'london': 1, 'is': 2, 'the': 1, 'capital': 1, 'of': 1, 'gb': 2, 'oxford': 1, 'a': 1, 'city': 1, 'in': 1}
IE_BOW:  {'dublin': 1, 'is': 2, 'the': 1, 'capital': 1, 'of': 1, 'ireland': 2, 'limerick': 1, 'a': 1, 'city': 1, 'in': 1}

V:  {'london': 1, 'is': 4, 'the': 2, 'capital': 2, 'of': 2, 'gb': 2, 'oxford': 1, 'a': 2, 'city': 2, 'in': 2, 'dublin': 1, 'ireland': 2, 'limerick': 1}
|V|:  13

Test document:  University of Limerick
Word: 'university' 	 wordConditionalProbGB: 0.0 	 wordConditionalProbIE: 0.0
Word: 'of' 	 wordConditionalProbGB: 0.07692307692307693 	 wordConditionalProbIE: 0.07692307692307693
Word: 'limerick' 	 wordConditionalProbGB: 0.0 	 wordConditionalProbIE: 0.07692307692307693
docProbGB = 0.16 	 docProbIE = 0.2
Inferred Class:  IE
--------------------------------------------------
Test document:  Univ